In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from itertools import chain

from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K


In [6]:
img_width = 128
img_height = 128
img_channels = 3

train_path = 'U_NET/train/'
test_path = 'U_NET/validation/'

In [7]:
train_ids = next(os.walk(train_path))[1]
test_ids = next(os.walk(test_path))[1]

In [8]:
X_train = np.zeros((len(train_ids), 128, 128, 3), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), 128, 128, 1), dtype=np.bool)

In [9]:

for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    
    path = train_path + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    img = resize(img, (128, 128), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((128, 128, 1), dtype=np.bool)
    
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (128, 128), mode='constant', preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
        Y_train[n] = mask 

100%|██████████| 670/670 [06:41<00:00,  1.67it/s]


In [10]:
X_test = np.zeros((len(test_ids), 128, 128, 3), dtype=np.uint8)
sizes_test = []

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = test_path + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:3]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (128, 128), mode='constant', preserve_range=True)
    X_test[n] = img

100%|██████████| 65/65 [00:02<00:00, 27.13it/s]


In [12]:
X_train = X_train/255
X_test = X_test/255

In [11]:
def iou_loss(y_true, y_pred):
    
    intersection = K.sum(K.abs(y_true + y_pred), axis = -1)
    union = K.sum((y_true, -1) + K.sum(y_pred, -1) - intersection)
    
    return (intersection +1) / (union + 1)

In [17]:
def model():
    
    inputs = Input((128, 128, 3))
    
    x = inputs
    
    c1 = Conv2D(16, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(x)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(64, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p2)
    c3 = Dropout(0.1)(c3)
    c3 = Conv2D(64, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c4 = Conv2D(128, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p3)
    c4 = Dropout(0.1)(c3)
    c4 = Conv2D(128, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c3)
    p4 = MaxPooling2D((2, 2))(c3)
    
    c5 = Conv2D(256, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(p4)
    c5 = Dropout(0.1)(c5)
    c5 = Conv2D(256, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c5)
    
    u6 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (2, 2), strides = (2, 2), padding = 'same')(u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)
    
    u7 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (2, 2), strides = (2, 2), padding = 'same')(u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)
    
    u8 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (2, 2), strides = (2, 2), padding = 'same')(u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    
    u9 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(16, (2, 2), strides = (2, 2), padding = 'same')(u8)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same')(c9)
    
    output = Conv2D(1, (1, 1), activation = 'sigmoid')(c9)
    
    model = Model(inputs = [inputs], outputs = [output])
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [iou_loss])
    
    return model

In [18]:
model = model()


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 32, 32, 32), (None, 64, 64, 32)]